In [6]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os

from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense

Most of the pictures are showing the subject in similar way. To prevent the model from overfitting we need to apply a random transformation to each picture.

In [7]:
datagen = ImageDataGenerator(rotation_range=40, #randomly rotate picture within [0-40] degrees
                            width_shift_range = 0.2, # Translate picture horizontally within [0, 0.2] is the fraction of the the original image width
                            height_shift_range = 0.2, # Translate picture vertically within [0, 0.2] is the fraction of the the original image width
                            rescale = 1/255, # Originally RGB values are within [0-255], rescale the data so that each data point is between 0 and 1. 
                            shear_range = 0.2, # Apply a shear transformation. ie __        ___
                                               #                                 |__| ---> /__/
                            zoom_range = 0.2, # Zoom randomly within the given range
                            horizontal_flip = True, # Flip half of the image randomly
                            fill_mode = 'nearest') # After the transformation some pixel will be empty. This option
                                                    # specifies what method should be used to fill those empty pixels.

Load an image and convert it into a numpy array

In [9]:
img = load_img('dogs-vs-cats/train/cats/cat.1000.jpg')
imgArray = img_to_array(img) # Convert image into a tensor
imgArray = imgArray.reshape((1,) + imgArray.shape) # (1, 3, 150, 150) Add another dimension (I couldn't understand this part)
                                            # maybe for classification purposes. 
                                            # This is a requirement for .flow()!

Apply the random transformation.

In [10]:
n = 0
for batch in datagen.flow(imgArray, 
                          batch_size = 1, # Using one transformation at a time.
                          save_to_dir = '/Users/uozdemir/iCloud Drive (Archive) - 1/Documents/GitHub/ULAS_OZDEMIR_AQM2020/Assignments/Week13/dogs-vs-cats/preview',
                          save_prefix = 'cat', 
                          save_format = 'jpeg'):
    n = n + 1
    if n >= 20: # Generate 20 transformed images
        break

Create the convNet. There are 3 layers of convolution layers with RELU activation.

In [11]:
model = Sequential()

model.add(Conv2D(32, (3,3), input_shape = (150, 150, 3))) #Using 32 filters with 3x3 kernel size. Size of the images are 150x150.
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size = (2,2))) # Each pool size is 2x2. This is used to reduce the dimension of the output.

model.add(Conv2D(32, (3,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size = (2,2)))


model.add(Conv2D(64, (3,3))) # The example uses 64 filters
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size = (2,2))) 
# Outputs will be a 3D feature map

Instructions for updating:
Colocations handled automatically by placer.


In [12]:
model.add(Flatten()) # Convert 3D matrix into 1D vectors
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))


In [13]:
model.compile(loss='binary_crossentropy', # This is a binary classification problem. Cross entropy is a suitable loss function.
              optimizer='rmsprop', #?
              metrics=['accuracy'])

In [16]:
batch_size = 400

train_datagen = ImageDataGenerator( # Generate training images
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

# this is the augmentation configuration we will use for testing:
# only rescaling
test_datagen = ImageDataGenerator(rescale=1./255) #Matrix elements are from 0-255. Scale elements so that they are within [0-1 range]

train_generator = train_datagen.flow_from_directory(
        'dogs-vs-cats/train',  # Target directory. Note that the code is looking for subfolders in train/ folder
        target_size=(150, 150),  # All images are 150x150
        batch_size=batch_size,
        class_mode='binary')  # This is a binary classification.


validation_generator = test_datagen.flow_from_directory( # Create validation genenerator by using similar settings
        'dogs-vs-cats/validation', #The code looks for a subfolders of validation/
        target_size=(150, 150),
        batch_size=batch_size,
        class_mode='binary')

Found 2000 images belonging to 2 classes.
Found 800 images belonging to 2 classes.


In [17]:
model.fit_generator(
        train_generator,
        steps_per_epoch=2000 // batch_size,
        epochs=10,
        validation_data=validation_generator,
        validation_steps=800 // batch_size)
model.save_weights('first_try.h5')  # save weights

Epoch 1/10
5/5 [==============================] - 31s 6s/step - loss: 0.6875 - accuracy: 0.6065 - val_loss: 0.6928 - val_accuracy: 0.5000
Epoch 2/10
5/5 [==============================] - 27s 5s/step - loss: 0.6876 - accuracy: 0.5655 - val_loss: 0.6657 - val_accuracy: 0.6450
Epoch 3/10
5/5 [==============================] - 27s 5s/step - loss: 0.6862 - accuracy: 0.5620 - val_loss: 0.6727 - val_accuracy: 0.6100
Epoch 4/10
5/5 [==============================] - 26s 5s/step - loss: 0.6710 - accuracy: 0.5815 - val_loss: 0.6581 - val_accuracy: 0.5475
Epoch 5/10
5/5 [==============================] - 27s 5s/step - loss: 0.6898 - accuracy: 0.5635 - val_loss: 0.6646 - val_accuracy: 0.6325
Epoch 6/10
5/5 [==============================] - 27s 5s/step - loss: 0.6873 - accuracy: 0.6115 - val_loss: 0.6708 - val_accuracy: 0.6650
Epoch 7/10
5/5 [==============================] - 27s 5s/step - loss: 0.6705 - accuracy: 0.6055 - val_loss: 0.6723 - val_accuracy: 0.5612
Epoch 8/10
5/5 [==================

We got 78% accuracy with only 10 epochs.

# Part 2: Bottleneck Features

In [ ]:
batch_size = 16
generator = datagen.flow_from_directory(
        'dogs-vs-cats/train',
        target_size = (150, 150),
        batch_size = batch_size,
        class_mode = None,  # No labels. Only batches of data will be used. Since the data is already ordered.
                        # There should be no mixing between batches
        shuffle = False)  # Data is ordered.
# save the output as a Numpy array
bottleneck_features_train = model.predict_generator(generator, 2000)

np.save(open('bottleneck_features_train.npy', 'wb'), bottleneck_features_train)

generator = datagen.flow_from_directory(
        'dogs-vs-cats/validation',
        target_size = (150, 150),
        batch_size = batch_size,
        class_mode = None,
        shuffle = False)
bottleneck_features_validation = model.predict_generator(generator, 800)
np.save(open('bottleneck_features_validation.npy', 'wb'), bottleneck_features_validation)

In [40]:
train_data = np.load(open('bottleneck_features_train.npy','rb'))
# the features were saved in order, so recreating the labels is easy
train_labels = np.array([0] * 16000 + [1] * 16000)
print(train_labels.shape,train_data.shape)
validation_data = np.load(open('bottleneck_features_validation.npy','rb'))
validation_labels = np.array([0] * 6400 + [1] * 6400)
model = Sequential()
# model.add(Flatten(input_shape=train_data.shape[1:]))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.fit(train_data, train_labels,
          epochs=10,
          batch_size=batch_size,
          validation_data=(validation_data, validation_labels))
model.save_weights('bottleneck_fc_model.h5')

(32000,) (32000, 1)
Train on 32000 samples, validate on 12800 samples
Epoch 1/10
32000/32000 [==============================] - 2s 67us/step - loss: 0.6936 - accuracy: 0.5016 - val_loss: 0.6933 - val_accuracy: 0.5000
Epoch 2/10
32000/32000 [==============================] - 2s 62us/step - loss: 0.6937 - accuracy: 0.4975 - val_loss: 0.6934 - val_accuracy: 0.5000
Epoch 3/10
32000/32000 [==============================] - 2s 62us/step - loss: 0.6934 - accuracy: 0.5018 - val_loss: 0.6932 - val_accuracy: 0.5000
Epoch 4/10
32000/32000 [==============================] - 2s 62us/step - loss: 0.6935 - accuracy: 0.4977 - val_loss: 0.6936 - val_accuracy: 0.5000
Epoch 5/10
32000/32000 [==============================] - 2s 62us/step - loss: 0.6933 - accuracy: 0.4989 - val_loss: 0.6932 - val_accuracy: 0.5000
Epoch 6/10
32000/32000 [==============================] - 2s 62us/step - loss: 0.6934 - accuracy: 0.4932 - val_loss: 0.6932 - val_accuracy: 0.5000
Epoch 7/10
32000/32000 [========================